In [1]:
!pip install "pillow == 11.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 81.2 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.3.0
    Uninstalling pillow-11.3.0:
      Successfully uninstalled pillow-11.3.0


In [2]:
# @title Install deps (run once)
!pip -q install "transformers>=4.45.0" accelerate bitsandbytes sentencepiece datasets --upgrade
!pip install specdec_bench
import time, math, torch, gc, threading
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer, set_seed
from threading import Thread
import sys, time, re, pandas as pd


print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 51.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
ERROR: Could not find a version that satisfies the requirement specdec_bench (from versions: none)
ERROR: No matching distribution found for specdec_bench
PyTorch: 2.8.0+cu126
CUDA available: True


In [3]:
#@title Imports, device, config
import time, statistics as stats, torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# `infer_device` helper (use HF's if present)
try:
    from transformers import infer_device
except Exception:
    def infer_device():
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = infer_device()
print("Device:", device)

# ==== Config (edit freely) ====
MODEL_ID = "HuggingFaceTB/SmolLM-1.7B"
ASSIST_ID = "HuggingFaceTB/SmolLM-135M"

PROMPT = "Hugging Face is an open-source company that can be summarized as "
MAX_NEW_TOKENS = 60
NUM_ASSISTANT_TOKENS = 2          # try 2, 4, 8, 16
PROMPT_LOOKUP_NUM_TOKENS = 5      # as in your snippet
REPEATS = 5

# Precision: fp16 on CUDA (T4/A100/L4); fp32 on CPU
DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32
print("dtype:", DTYPE)


Device: cuda
dtype: torch.float16


In [4]:
#@title Load models/tokenizer
tok = AutoTokenizer.from_pretrained(MODEL_ID)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=DTYPE).to(device).eval()
assist = AutoModelForCausalLM.from_pretrained(ASSIST_ID, torch_dtype=DTYPE).to(device).eval()

# Silence "pad_token_id" messages by setting it on the configs
model.config.pad_token_id = model.config.eos_token_id
assist.config.pad_token_id = assist.config.eos_token_id

print("Loaded models.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Loaded models.


In [5]:
#@title  Load dataset
import time, math
import torch
import pandas as pd
from datasets import load_dataset

# existing model + tokenizer
target_model = model
tok          = tok
draft_model = assist

if tok.pad_token_id is None and tok.eos_token_id is not None:
    tok.pad_token_id = tok.eos_token_id

# --- Load a small sample from SmolLM-Corpus ---
# Pick one subset: "fineweb-edu-dedup" or "cosmopedia-v2"
SUBSET   = "fineweb-edu-dedup"
SAMPLES  = 100
CTX_LEN  = 128      # prompt length (tokens)
GEN_TOKS = 64       # how many tokens to generate for timing

ds_stream = load_dataset("HuggingFaceTB/smollm-corpus",
                         SUBSET, split="train", streaming=True)
texts = []
for i, ex in enumerate(ds_stream):
    texts.append(ex["text"])
    if i+1 >= SAMPLES:
        break

# --- Build prompts from the first CTX_LEN tokens of each text ---
prompts = []
for txt in texts:
    ids = tok(txt, return_tensors="pt", add_special_tokens=False).input_ids[0]
    ids = ids[:CTX_LEN]
    prompts.append(tok.decode(ids))

len(prompts)

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/234 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4303 > 2048). Running this sequence through the model will result in indexing errors


100

In [6]:
#@title  Check model perplexity
# Compute per-sample NLL (nats/token) and perplexity using up to 512 tokens from each text
MAX_EVAL_TOKS = 512
nll_list = []

for txt in texts:
    enc = tok(txt, return_tensors="pt", add_special_tokens=False, truncation=True, max_length=MAX_EVAL_TOKS)
    if torch.cuda.is_available():
        enc = {k: v.cuda() for k,v in enc.items()}
    with torch.no_grad():
        out = target_model(**enc, labels=enc["input_ids"])
        loss = float(out.loss.item())  # mean cross-entropy over tokens (nats)
    nll_list.append(loss)

ppl = math.exp(sum(nll_list)/len(nll_list))
print(f"Perplexity on SmolLM-Corpus/{SUBSET} (N={SAMPLES}): {ppl:.2f}   [avg nll={sum(nll_list)/len(nll_list):.4f} nats/token]")


Perplexity on SmolLM-Corpus/fineweb-edu-dedup (N=100): 9.15   [avg nll=2.2140 nats/token]


In [36]:
#@title  Runtime evaluation
def to_device(batch):
    if torch.cuda.is_available():
        return {k: v.cuda() for k,v in batch.items()}
    return batch

# ---- Baseline (greedy) ----
base_times = []
base_out_ids = []

for p in prompts:
    inputs = tok(p, return_tensors="pt")
    inputs = to_device(inputs)
    if torch.cuda.is_available(): torch.cuda.synchronize()
    t0 = time.perf_counter()
    out = target_model.generate(
        **inputs,
        max_new_tokens=GEN_TOKS,
        do_sample=False,
        use_cache=True,
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id,
    )
    if torch.cuda.is_available(): torch.cuda.synchronize()
    t1 = time.perf_counter()
    base_times.append(t1 - t0)

    # store only the generated suffix for parity check
    gen_ids = out[0, inputs["input_ids"].shape[1]:]
    base_out_ids.append(gen_ids.cpu())

base_total = sum(base_times)
print(f"Baseline total time: {base_total:.2f}s  | avg/sample: {base_total/len(prompts):.4f}s")

# ---- Speculative (assistant_model) ----
NUM_ASSIST = 4  # sweep 4/8/16/32 later
spec_times = []
spec_out_ids = []

for p in prompts:
    inputs = tok(p, return_tensors="pt")
    inputs = to_device(inputs)
    if torch.cuda.is_available(): torch.cuda.synchronize()
    t0 = time.perf_counter()
    out = target_model.generate(
        **inputs,
        max_new_tokens=GEN_TOKS,
        do_sample=False,
        use_cache=True,
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id,
        assistant_model=draft_model,
        num_assistant_tokens=NUM_ASSIST,
        prompt_lookup_num_tokens=20,
    )
    if torch.cuda.is_available(): torch.cuda.synchronize()
    t1 = time.perf_counter()
    spec_times.append(t1 - t0)

    gen_ids = out[0, inputs["input_ids"].shape[1]:]
    spec_out_ids.append(gen_ids.cpu())

spec_total = sum(spec_times)
print(f"Speculative total time: {spec_total:.2f}s | avg/sample: {spec_total/len(prompts):.4f}s")
print(f"Speedup (total): {base_total/spec_total:.2f}×")


Baseline total time: 171.00s  | avg/sample: 1.7100s
Speculative total time: 151.47s | avg/sample: 1.5147s
Speedup (total): 1.13×
